In [1]:
!pip install mediapipe opencv-python

In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a) #first
    b = np.array(b) #second
    c = np.array(c) #third
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

In [ ]:
cap = cv2.VideoCapture(0)

#CUrl Counter variables
counter_right = 0
stage_right = None
counter_left = 0
stage_left = None

#setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # RECOLOR IMAGE to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #make detection
        result = pose.process(image)
        
        #RECOLOR back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #extract landmark
        try:
            landmarks = result.pose_landmarks.landmark
            
            #get cordinates
            #RIGHT
            shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist_right = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            #LEFT
            shoulder_left = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow_left = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist_left = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            #calculate angle
            angle_right = calculate_angle(shoulder_right, elbow_right, wrist_right)
            angle_left = calculate_angle(shoulder_left, elbow_left,wrist_left)
            
            #visualize
            cv2.putText(image, str(angle_right),
                            tuple(np.multiply(elbow_right,[640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                       )
            cv2.putText(image, str(angle_left),
                            tuple(np.multiply(elbow_left,[640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                       )
            
            #curl counter logic
            if angle_right > 160:
                stage_right = "down"
            if angle_right < 30 and stage_right == "down":
                stage_right="up"
                counter_right += 1
            if angle_left > 160:
                stage_left = "down"
            if angle_left < 30 and stage_left == "down":
                stage_left = "up"
                counter_left += 1
                
        except:
            pass
        
        #render curl counter
        #setup status box
        cv2.rectangle(image,(0,0),(325,73), (245,117,16),-1)
        cv2.rectangle(image,(325,0),(650,73), (212,41,58),-1)
        
        #RIGHT
        #rep data right
        cv2.putText(image, "REPS &", (15,12),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter_right), (10,60),
                   cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        #stage data right
        cv2.putText(image, "STAGE RIGHT", (90,12),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage_right, (90,60),
                   cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        #LEFT
        #rep data left
        cv2.putText(image, "REPS &", (340,12),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter_left), (340,60),
                   cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        #stage data left
        cv2.putText(image, "STAGE LEFT", (415,12),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage_left, (420,60),
                   cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        #render detections
        mp_drawing.draw_landmarks(image, result.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(245,66,280), thickness=2, circle_radius=2)
                                 )

        bigger = cv2.resize(image, (1280, 720))
        cv2.imshow('Gym Repetisi', bigger)
        
        # Menangkap tombol keyboard
        key = cv2.waitKey(1)

        # Jika tombol 'r' atau 'l' ditekan, reset perhitungan
        if key == ord('r'):
            counter_right = 0
            stage_right = None
        if key == ord('l'):
            counter_left = 0
            stage_left = None
            
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()